
#  Optimización del sistema Encicla en Medellín
* Felipe Miranda Arboleda
* Emilio Porras Mejía

## Modelo inicial y obtención de datos

In [ ]:
%pip install folium pandas

In [ ]:
import folium
import pandas as pd

In [ ]:


# Cargar el archivo CSV
df = pd.read_csv('Estaciones_EnCicla_AMVA_20250713.csv', header=None)

# Dividir la primera columna en múltiples columnas usando comas
split_columns = df[0].str.split(',', expand=True)

# Extraer latitud y longitud
df['Latitude'] = split_columns.iloc[:, -1]
df['Longitude'] = df[1]

# Eliminar las columnas originales
df = df.drop(columns=[0, 1])

# Renombrar las columnas
df.columns = df.iloc[0]
df = df[1:]
df.reset_index(drop=True, inplace=True)



print(df.head())

0               DIRECCION                                    Ayuda Ubicación  \
0       CALLE 82A # 52-29  Frente a la entrada principal del Centro de De...   
1         CALLE 73 # 52-2  Bajo el viaducto de la estación Universidad de...   
2     CARRERA 52 # 67A-13  Frente a la fachada principal de Ruta N sobre ...   
3  CARRERA 77A # 63 - 100             Diagonal al Colegio Mayor de Antioquia   
4    CALLE 59A # 64C - 44  Portería de la Iguaná con Carrera 65 de la Uni...   

0               MUNICIPIO    TIPO TOTAL ANCLAJES  Georeferenciación  #  \
0    Zona 1 - Nororiental  MANUAL             15   6,27658;-75,5648  1   
1    Zona 1 - Nororiental  MANUAL             33  6,26939;-75,56577  2   
2    Zona 1 - Nororiental  AUTOM.             40  6,26518;-75,56634  3   
3  Zona 2 - Noroccidental  MANUAL             21  6,27197;-75,59131  4   
4  Zona 2 - Noroccidental  MANUAL             33    6,25971;-75,579  5   

0  NOMBRE ESTACION  
0          Moravia  
1      Universidad  
2          

In [ ]:

df[['Latitude', 'Longitude']] = df['Georeferenciación'].str.split(';', expand=True)


df['Latitude'] = df['Latitude'].str.replace(',', '.').astype(float)
df['Longitude'] = df['Longitude'].str.replace(',', '.').astype(float)

print(df[['Latitude', 'Longitude']].head())

0  Latitude  Longitude
0   6.27658  -75.56480
1   6.26939  -75.56577
2   6.26518  -75.56634
3   6.27197  -75.59131
4   6.25971  -75.57900


In [ ]:

# Crear un mapa base centrado en Medellín
medellin_map = folium.Map(location=[6.2442, -75.5812], zoom_start=12)

# Agregar marcadores para cada estación
for _, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['NOMBRE ESTACION']  # La columna 'NOMBRE ESTACION' contiene los nombres de las estaciones
    ).add_to(medellin_map)

# Guardar el mapa en un archivo HTML
medellin_map.save('medellin_map.html')

# Mostrar el mapa en el notebook
medellin_map

In [11]:
from math import radians, sin, cos, sqrt, atan2
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    # Convertir latitud y longitud de grados a radianes
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Fórmula de Haversine
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    radius = 6371  # Radio de la Tierra en kilómetros
    return radius * c  # Retorna la distancia en kilómetros

# Generar la matriz de distancias
num_locations = len(df)
distance_matrix = np.zeros((num_locations, num_locations))

for i in range(num_locations):
    for j in range(num_locations):
        distance_matrix[i, j] = haversine(df.iloc[i]['Latitude'], df.iloc[i]['Longitude'],
                                          df.iloc[j]['Latitude'], df.iloc[j]['Longitude'])

# Convertir la matriz de distancias a un DataFrame para mejor legibilidad
distance_df = pd.DataFrame(distance_matrix, columns=df['NOMBRE ESTACION'], index=df['NOMBRE ESTACION'])

print(distance_df.head())

NOMBRE ESTACION   Moravia  Universidad    Ruta N   Robledo  Campus Nacional  \
NOMBRE ESTACION                                                               
Moravia          0.000000     0.806648  1.278999  2.974622         2.445867   
Universidad      0.806648     0.000000  0.472351  2.837467         1.815753   
Ruta N           1.278999     0.472351  0.000000  2.861345         1.525801   
Robledo          2.974622     2.837467  2.861345  0.000000         1.926077   
Campus Nacional  2.445867     1.815753  1.525801  1.926077         0.000000   

NOMBRE ESTACION  Dian Alpujarra  Distrito Creativo  Plaza de La Libertad  \
NOMBRE ESTACION                                                            
Moravia                3.784073           4.733480              3.840337   
Universidad            2.981270           3.927417              3.045773   
Ruta N                 2.512285           3.455524              2.583924   
Robledo                3.813567           4.672503              3.